In [204]:
!pip install fastai 

In [255]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image
import PIL.image

  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [256]:
cd

/content


In [0]:
#!git clone https://github.com/fastai/fastai.git

In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'


In [0]:
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch


In [260]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:80: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL:
Core version: 4.0.0
Pillow version: 5.0.0
  warnings.warn(str(v), RuntimeWarning)
[autoreload of PIL.Image failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: The _imaging extension was built for another version of Pillow or PIL:
Core version: 4.0.0
Pillow version: 5.0.0
]
[autoreload of PIL.JpegImagePlugin failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: module 'PIL.Image' has no attribute 'register_extensions'
]
[autoreload of PIL.TiffImagePlugin failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/

In [0]:
!pip install -q keras
import keras

In [262]:
cd

/content


In [263]:
ls

datalab/  dogbreed/


In [264]:
cd /usr/local/lib/python3.6/dist-packages/fastai

/usr/local/lib/python3.6/dist-packages/fastai


In [265]:
ls

column_data.py   initializers.py     lm_rnn.py   plots.py      structured.py
conv_learner.py  __init__.py         losses.py   __pycache__/  torch_imports.py
core.py          io.py               metrics.py  rnn_reg.py    transforms.py
dataloader.py    layer_optimizer.py  model.py    rnn_train.py  utils.py
dataset.py       layers.py           models/     set_spawn.py  weights/
imports.py       learner.py          nlp.py      sgdr.py       weights.tgz


In [0]:
#!wget http://files.fast.ai/models/weights.tgz

In [0]:
#!tar -xvzf weights.tgz

In [268]:
cd

/content


In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression

In [270]:
cd

/content


In [271]:
ls

datalab/  dogbreed/


In [0]:
#mkdir dogbreed

In [273]:
cd dogbreed

/content/dogbreed


In [274]:
!pip install Pillow==4.0.0 
!pip install PIL 
!pip install Image 


  Using cached Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: Pillow 5.0.0
    Uninstalling Pillow-5.0.0:
      Successfully uninstalled Pillow-5.0.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [275]:
ls

labels.csv      sample_submission.csv      test/     tmp/    train.zip
labels.csv.zip  sample_submission.csv.zip  test.zip  train/


In [0]:
#!pip install kaggle-cli

In [0]:
#!kg config -g -u 'pursh2002' -p 'MyCar1973@' -c 'dog-breed-identification'

In [0]:
#!kg download

In [0]:
#!unzip labels.csv.zip

In [0]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.imports import *

In [0]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
arch = resnext101_64


In [283]:
ls

labels.csv      sample_submission.csv      test/     tmp/    train.zip
labels.csv.zip  sample_submission.csv.zip  test.zip  train/


In [284]:
pwd

'/content/dogbreed'

In [0]:
PATH = "/content/dogbreed/"
sz=224
bs=58
arch = resnext101_64


In [0]:
lable_csv = f'{PATH}labels.csv'
n=len(list(open(lable_csv)))-1
val_idxs=get_cv_idxs(n)

In [287]:
df = pd.read_csv('labels.csv')
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [288]:
df.pivot_table(index='breed',aggfunc=len).sort_values('id',ascending=False)

,id
breed,
scottish_deerhound,126
maltese_dog,117
afghan_hound,116
entlebucher,115
bernese_mountain_dog,114
shih-tzu,112
great_pyrenees,111
pomeranian,111
basenji,110


In [0]:
# data agumantations 
tfms = tfms_from_model(arch,sz,aug_tfms=transforms_side_on,max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'train',f'{PATH}labels.csv',test_name='test',val_idxs=val_idxs,suffix='.jpg',tfms=tfms,bs=bs)

In [290]:
len(val_idxs)

2044

In [291]:
n

10222

In [292]:
fn= PATH+data.trn_ds.fnames[0];fn

'/content/dogbreed/train/001513dfcb2ffafc82cccf4d8bbaba97.jpg'

In [294]:
!pip install Pillow 
!pip install PIL 
!pip install Image 

  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [295]:
from PIL.Image import Image
img = PIL.Image.open(fn);img

AttributeError: ignored

In [296]:
print (Image)

<class 'PIL.Image.Image'>


In [297]:
import  PIL as pillow

from PIL.Image import Image
img = PIL.Image.open(fn)
img

AttributeError: ignored

In [298]:
img.size

NameError: ignored

In [299]:
size_d={k:PIL.Image.open(PATH+k).size for k in data.trn_ds.fnames}

AttributeError: ignored

In [300]:
row_sz,col_sz=list(zip(*size_d.values()))

NameError: ignored

In [301]:
row_sz=np.array(row_sz)
col_sz=np.array(col_sz)

NameError: ignored

In [302]:
row_sz[:5]

NameError: ignored

In [303]:
plt.hist(row_sz)

NameError: ignored

In [304]:
plt.hist(row_sz[row_sz<1000])

NameError: ignored

In [0]:
#initial model 
def get_data(sz,bs):
  tfms = tfms_from_model(arch,sz,aug_tfms=transforms_side_on,max_zoom=1.1)
  data=ImageClassifierData.from_csv(PATH, 'train',f'{PATH}labels.csv',test_name='test',num_workers=4,val_idxs=val_idxs,suffix='.jpg',tfms=tfms,bs=bs)
  return data if sz>300 else data.resize(340,'tmp')

In [306]:
data = get_data(sz,bs)


HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

In [307]:
learn = ConvLearner.pretrained(arch,data, precompute=True)

100%|██████████| 179/179 [05:14<00:00,  1.76s/it]


In [308]:
learn.fit(1e-2,5)

HBox(children=(IntProgress(value=0, description='Epoch', max=5), HTML(value='')))

[0.      0.91475 0.41482 0.90374]
[1.      0.43515 0.35111 0.91188]
[2.      0.29851 0.30295 0.91283]
[3.      0.2412  0.3499  0.91619]
 75%|███████▌  | 106/141 [00:02<00:00, 45.37it/s, loss=0.204]

[4.      0.19597 0.28408 0.91667]



In [0]:
learn = ConvLearner.pretrained(arch, data, precompute=True,ps=0.5)

In [310]:
learn.fit(1e-2,2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[0.      1.14753 0.44911 0.90182]
[1.      0.54667 0.31257 0.92146]



In [0]:
learn.precompute=False

In [312]:
learn.fit(1e-2,5,cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=5), HTML(value='')))

[0.      0.45589 0.26385 0.92816]
[1.      0.42145 0.25324 0.92768]
[2.      0.38815 0.2406  0.93199]
 43%|████▎     | 60/141 [01:49<02:27,  1.82s/it, loss=0.377]

[3.      0.36226 0.23389 0.93056]
[4.      0.33665 0.23508 0.92577]



In [313]:
learn.fit(1e-2,3,cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[0.      0.32488 0.2288  0.92816]
[1.      0.30236 0.2268  0.92768]
[2.      0.27472 0.22514 0.9272 ]



In [0]:
learn.save('224_pre')


In [0]:
learn.load('224_pre')

In [316]:
learn.set_data(get_data(299,bs))
learn.freeze()


HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

In [317]:
learn.fit(1e-2,3,cycle_len=1)


HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[0.      0.30297 0.22079 0.9272 ]
[1.      0.28582 0.21704 0.9296 ]
[2.      0.2616  0.21333 0.93343]



In [319]:
learn.fit(1e-2,3,cycle_len=1,cycle_mult=2)


HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[0.      0.24699 0.20844 0.93151]
[1.      0.2514  0.21616 0.9296 ]
[2.      0.22306 0.21041 0.93295]
 43%|████▎     | 60/141 [03:20<04:30,  3.34s/it, loss=0.222]

[3.      0.23865 0.21098 0.93056]
[4.      0.21477 0.20804 0.93439]
[5.      0.19727 0.20743 0.93199]
 59%|█████▉    | 83/141 [04:30<03:09,  3.26s/it, loss=0.194]

[6.      0.18372 0.2064  0.93391]



In [320]:
log_preds,y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds,y),metrics.log_loss(y,probs)


(0.9383561643835616, 0.19883894320245182)

In [0]:
learn.save('299_pre')
learn.load('299_pre')


In [323]:
learn.fit(1e-2,1,cycle_len=2)



HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[0.      0.20062 0.21159 0.93295]
[1.      0.18526 0.20883 0.92864]



In [0]:
learn.save('299_pre')

In [325]:
log_preds,y = learn.TTA()
probs= np.exp(log_preds)
accuracy(log_preds,y),metrics.log_loss(y,probs)



(0.9339530332681018, 0.20234849747546)